In [ ]:
# STEP 1: Clone YOLOv5 and install dependencies
!git clone https://github.com/ultralytics/yolov5
%cd yolov5
!pip install -r requirements.txt
!pip install deep_sort_realtime opencv-python-headless matplotlib

# STEP 2: Upload 'input.mp4'
from google.colab import files
uploaded = files.upload()  # Upload your 'input.mp4'

# STEP 3: Imports
import cv2
import torch
import numpy as np
from yolov5.models.common import DetectMultiBackend
from yolov5.utils.general import non_max_suppression, scale_boxes
from yolov5.utils.augmentations import letterbox
from deep_sort_realtime.deepsort_tracker import DeepSort
from collections import defaultdict
from matplotlib import pyplot as plt

# STEP 4: Setup
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
model = DetectMultiBackend('yolov5s.pt', device=device)
model.eval()

# Video setup
video_path = 'input.mp4'
cap = cv2.VideoCapture(video_path)
width = int(cap.get(cv2.CAP_PROP_FRAME_WIDTH))
height = int(cap.get(cv2.CAP_PROP_FRAME_HEIGHT))
fps = cap.get(cv2.CAP_PROP_FPS)

fourcc = cv2.VideoWriter_fourcc(*'mp4v')
out = cv2.VideoWriter('output.mp4', fourcc, fps, (width, height))

# DeepSORT & Tracking Variables
deepsort = DeepSort()
track_history = defaultdict(list)
counted_ids_in = set()
counted_ids_out = set()
car_count_in = 0
car_count_out = 0

# Lines for counting
entry_line_y = int(height * 0.4)
exit_line_y = int(height * 0.6)

# Classes to track (car=2, motorcycle=3, bus=5, truck=7)
VEHICLE_CLASSES = [2, 3, 5, 7]

frame_count = 0
while cap.isOpened():
    ret, frame = cap.read()
    if not ret:
        break
    frame_count += 1

    # Preprocess
    img = letterbox(frame, new_shape=640)[0]
    img = img[:, :, ::-1].transpose(2, 0, 1)
    img = np.ascontiguousarray(img)
    img_tensor = torch.from_numpy(img).to(device).float() / 255.0
    img_tensor = img_tensor.unsqueeze(0)

    # Inference
    with torch.no_grad():
        pred = model(img_tensor, augment=False)
        pred = non_max_suppression(pred, conf_thres=0.3, iou_thres=0.4)[0]

    detections = []
    if pred is not None and len(pred):
        pred[:, :4] = scale_boxes(img_tensor.shape[2:], pred[:, :4], frame.shape).round()
        for *xyxy, conf, cls in pred:
            if int(cls.item()) in VEHICLE_CLASSES:
                x1, y1, x2, y2 = map(int, xyxy)
                w, h = x2 - x1, y2 - y1
                if w > 0 and h > 0:  # Only valid boxes
                    detections.append(([x1, y1, w, h], conf.item(), 'vehicle'))

    # DeepSORT Tracking
    tracks = deepsort.update_tracks(detections, frame=frame)
    for track in tracks:
        if not track.is_confirmed():
            continue

        track_id = track.track_id
        l, t, r, b = map(int, track.to_ltrb())
        cx = int((l + r) / 2)
        cy = int((t + b) / 2)

        # History
        track_history[track_id].append((cx, cy, frame_count))
        if len(track_history[track_id]) > 2:
            track_history[track_id].pop(0)

        # Speed in mph
        speed = 0
        if len(track_history[track_id]) == 2:
            (x1, y1, f1), (x2, y2, f2) = track_history[track_id]
            dist_px = np.linalg.norm([x2 - x1, y2 - y1])
            time_elapsed = (f2 - f1) / fps
            if time_elapsed > 0:
                speed = (dist_px / time_elapsed) * 0.0447  # px/frame to mph

        # Count logic
        if len(track_history[track_id]) == 2:
            (, y_prev, _), (, y_curr, _) = track_history[track_id]
            if y_prev < entry_line_y <= y_curr and track_id not in counted_ids_in:
                car_count_in += 1
                counted_ids_in.add(track_id)
            elif y_prev > exit_line_y >= y_curr and track_id not in counted_ids_out:
                car_count_out += 1
                counted_ids_out.add(track_id)

        # Draw vehicle box and speed
        cv2.rectangle(frame, (l, t), (r, b), (0, 255, 0), 2)
        cv2.putText(frame, f'ID:{track_id} {speed:.1f} mph', (l, t - 10),
                    cv2.FONT_HERSHEY_SIMPLEX, 0.5, (255, 0, 0), 2)

    # Draw lines
    cv2.line(frame, (0, entry_line_y), (width, entry_line_y), (255, 0, 255), 2)
    cv2.line(frame, (0, exit_line_y), (width, exit_line_y), (0, 255, 255), 2)

    # Count display
    cv2.putText(frame, f'Incoming: {car_count_in}', (10, 30),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 255, 0), 3)
    cv2.putText(frame, f'Outgoing: {car_count_out}', (10, 70),
                cv2.FONT_HERSHEY_SIMPLEX, 1, (0, 0, 255), 3)

    out.write(frame)

cap.release()
out.release()
cv2.destroyAllWindows()
print("✅ Done! Download your 'output.mp4' from the sidebar.")

# Download output
from google.colab import files
files.download('output.mp4')